In [21]:
import time
from multiverse import MeasurementPlaneClient

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import time
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import time
from threading import Thread
import random

class DashPlotting:
    def __init__(self, channels):
        self.channels = channels
        self.channel_data = {channel: [] for channel in channels}
        self.timestamps = []
        self.start_time = None
        # Initialize Dash app
        self.app = dash.Dash(__name__)

        # Define the layout
        self.app.layout = html.Div([
            dcc.Graph(id='live-graph'),
            dcc.Interval(
                id='graph-update',
                interval=1000,  # Update every second
                n_intervals=0
            )
        ])

        # Set up the callback to update the graph
        @self.app.callback(
            Output('live-graph', 'figure'),
            [Input('graph-update', 'n_intervals')]
        )
        def update_graph_live(n):
            fig = go.Figure()

            # Limit to the last 100 data points
            timestamps_to_plot = self.timestamps[-500:]

            # Update the traces for each channel
            for channel in self.channels:
                channel_data_to_plot = self.channel_data[channel][-500:]  # Only last 100 points
                fig.add_trace(go.Scatter(x=timestamps_to_plot, y=channel_data_to_plot, mode='lines', name=f'Channel {channel}'))

            fig.update_layout(
                title={
                    "text": "Count rates of timetaggers channels",
                    "font": {"size": 24}  # Set title font size
                },
                xaxis={
                    "title": {"text": "Time (seconds)", "font": {"size": 18}},  # X-axis title font size
                    "tickfont": {"size": 14}  # X-axis tick font size
                },
                yaxis={
                    "title": {"text": "Rate/s", "font": {"size": 18}},  # Y-axis title font size
                    "tickfont": {"size": 14}  # Y-axis tick font size
                },
                legend={"font": {"size": 16}}  # Legend font size
            )

            return fig
        
    def start_server(self):
        # Start the Dash app server
        self.app.run_server(debug=True, use_reloader=False, inline_callbacks=False)  # Turn off reloader to avoid double execution

    def on_result_callback(self, result):
        #print(f"New Result Received: {result}")
        if isinstance(result, list) and result == ['EOF_results']:
            print("End of results received.")
            return  # Stop processing further since this is an EOF marker
        
        if result:
            result = result[0]  # Assuming result is a list of dicts
            
            # Initialize start time on first result
            if self.start_time is None:
                self.start_time = time.time()

            # Append new rates for each channel
            for channel in self.channels:
                self.channel_data[channel].append(result.get(channel, random.uniform(1, 10)))  # Append random value for example
            
            
            elapsed_time = time.time() - self.start_time
            self.timestamps.append(elapsed_time)

           
def on_completion_callback(result):
    print(result)
    pass


BROKER_URL = "http://localhost:5672/"

mpClient = MeasurementPlaneClient(BROKER_URL)

In [24]:

# Step 1: Get the capabilities published by agents
capabilities = mpClient.get_capabilities(capability_type="measure")
#print("Available Capabilities:", capabilities)
for id, cap in capabilities.items():
    print(f"{id}:{cap['label']}")


0:time tags measurement capability
1:count rate measurement capability


In [25]:
capability_id = 1
capability = capabilities[capability_id]
channels =  '1|2|3'
plotter = DashPlotting(channels.split('|'))
plotter.start_server()

In [28]:
# Step 3: Instantiate a specification of a measurement from the capability
measurement = mpClient.create_measurement(capability)
print(f"Created Measurement: {measurement.specification_message}")

# Step 4: Configure the measurement with parameters
#schedule = "now | 2024-11-15 22:50:00 | stream "  # Replace with your schedule: `now / period`, `start ... stop / period`, etc.
schedule = "now | 2024-11-21 22:01:00 |stream"
parameters = {
    "channels": channels
}
measurement.configure(
    schedule=schedule,
    parameters=parameters,
    stream_results=False,  # Stream results
    redirect_to_storage=True,  # Redirect to storage
    result_callback=plotter.on_result_callback,  # Callback function for new results
    completion_callback=on_completion_callback  # Callback function for measurement completion
)
print(f"Configured Measurement Specification: {measurement.specification_message}")

# Step 5: Send the Measurement
mpClient.send_measurement(measurement)

Created Measurement: {'label': 'count rate measurement capability', 'endpoint': '/multiverse/qnet/tt/Alice', 'capabilityName': 'count_rate_measurement_capability', 'parameters_schema': {'type': 'object', 'properties': {'channels': {'type': 'string', 'description': "String of channel numbers separated by '|' to measure count rates from"}}}, 'resultSchema': {'type': 'object', 'properties': {'type': 'object', 'patternProperties': {'^[0-9]+$': {'type': 'number', 'description': 'The calculated count rate for the channel'}}}}, 'timestamp': '2024-11-04 11:52:40.50', 'nonce': '12345', 'metadata': None, 'specification': 'measure'}
Configured Measurement Specification: {'label': 'count rate measurement capability', 'endpoint': '/multiverse/qnet/tt/Alice', 'capabilityName': 'count_rate_measurement_capability', 'parameters_schema': {'type': 'object', 'properties': {'channels': {'type': 'string', 'description': "String of channel numbers separated by '|' to measure count rates from"}}}, 'resultSche

In [29]:
# Step 6: Interrupt the measurement
mpClient.interrupt_measurement(measurement)
print(f"Measurement Interrupted")

Measurement Interrupted
